# **Несбалансированные классы**

Давайте порешаем еще одну финансовую задачку: будем предсказывать, возьмет клиент банка кредит или нет.

Описание датасета:

1. age - возраст (numeric)

2. job - вид работы (categorical: "admin.","blue-collar","entrepreneur" housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")

3. marital - супружеский статус (categorical:"divorced","married","single","unknown"; note: "divorced" means divorced or widowed)

4. education - уровень образования (categorical: "basic.4y", "basic.6y", "basic.9y", "high.school", "illiterate", "professional.course", "university.degree",unknown")

5. default - еcть ли уже кредит? (categorical: "no","yes","unknown")

6. housing - есть ли жилищный кредит? (categorical: "no","yes","unknown")

7. loan - есть ли кредит для личных целей? (categorical: "no","yes","unknown")

8. contact - контактный вид связи (categorical: "cellular","telephone")

9. month: месяц последнего контакта в году (categorical: "jan", "feb", "mar", ..., "nov", "dec")

10. day_of_week - последний контактный день недели (categorical: "mon","tue","wed","thu","fri..)

11. duration - длительность последнего контакта в секундах (numeric).

12. campaign - количество контактов, выполненных во время этой кампании для этого клиента (numeric, includes last contact)

13. Pdays - количество дней, прошедших с момента последнего контакта с клиентом из предыдущей кампании (numeric; 999 means client was not previously contacted)

14. previous - количество контактов, выполненных до этой кампании и для этого клиента (numeric)

15. poutcome - результат предыдущей маркетинговой кампании (categorical: "unknown","other","failure","success")

16. Emp.var.rate - коэффициент изменения занятости - квартальный показатель (numeric)

17. Cons.price.idx - индекс потребительских цен - месячный индикатор (numeric)

18. Cons.conf.idx - индекс потребительского доверия - месячный показатель (numeric)

19. Euribor3m - 3-месячная ставка, дневной индикатор (numeric)

20. Nr.employed - количество работников, квартальный показатель (numeric)
:
21. y (target) - подписал ли клиент срочный вклад (binary: "yes","no")


# 1. Импортируем нужные библиотеки

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# 2. Загружаем наш датасет

In [5]:
bank_addit_df = pd.read_csv('bank-additional-full.csv', sep=';')
bank_addit_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# 3. Оценка данных

In [6]:
# выведем размерность нашего датасета:
bank_addit_df.shape

(41188, 21)

In [7]:
# выведем типы данных:
bank_addit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [8]:
# выведем основные статистические показатели:
bank_addit_df.describe().round(3)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.000,41188.000,41188.000,41188.000,41188.000,41188.000,41188.000,41188.000,41188.000,41188.000
mean,40.024,258.285,2.568,962.475,0.173,0.082,93.576,-40.503,3.621,5167.036
std,10.421,259.279,2.770,186.911,0.495,1.571,0.579,4.628,1.734,72.252
min,17.000,0.000,1.000,0.000,0.000,-3.400,92.201,-50.800,0.634,4963.600
25%,32.000,102.000,1.000,999.000,0.000,-1.800,93.075,-42.700,1.344,5099.100
50%,38.000,180.000,2.000,999.000,0.000,1.100,93.749,-41.800,4.857,5191.000
75%,47.000,319.000,3.000,999.000,0.000,1.400,93.994,-36.400,4.961,5228.100
max,98.000,4918.000,56.000,999.000,7.000,1.400,94.767,-26.900,5.045,5228.100


# 4. Feature Engineering:

## 4.1.  Обработка категориальных признаков и создание новых признаков из уже существующих

In [9]:
print('age: ', bank_addit_df.age.unique(), '\n',
      'marital: ', bank_addit_df.marital.unique(), '\n',
      'default: ', bank_addit_df.default.unique(), '\n',
      'housing: ', bank_addit_df.housing.unique(), '\n',
      'loan: ', bank_addit_df.loan.unique(), '\n',
      'contact: ', bank_addit_df.contact.unique(), '\n',
      'month: ', bank_addit_df.month.unique(), '\n',
      'poutcome: ', bank_addit_df.poutcome.unique(), '\n',
      'y: ', bank_addit_df.y.unique()
      )

age:  [56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44
 42 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67
 73 88 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91
 86 98 94 84 92 89] 
 marital:  ['married' 'single' 'divorced' 'unknown'] 
 default:  ['no' 'unknown' 'yes'] 
 housing:  ['no' 'yes' 'unknown'] 
 loan:  ['no' 'yes' 'unknown'] 
 contact:  ['telephone' 'cellular'] 
 month:  ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep'] 
 poutcome:  ['nonexistent' 'failure' 'success'] 
 y:  ['no' 'yes']


In [10]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

def LabelEncoder(data, feature):
  data[feature] = label_encoder.fit_transform(data[feature])

  return(data)

In [11]:
from sklearn.preprocessing import OneHotEncoder

def One_Hot_Encoder(data, feature):
  encoded_columns = pd.get_dummies(data[feature])
  return(encoded_columns)

In [12]:
def change_type(data, feature):
  data[feature] = data[feature].astype(int)

  return(data)

In [13]:
# Поработаем с признаком 'age' (возраст):
def map_age(age):
  if age == '0-17':
    return 0
  elif age == '18-25':
    return 1
  elif age == '26-35':
    return 2
  elif age == '36-45':
    return 3
  elif age == '46-50':
    return 4
  elif age == '51-55':
    return 5
  else:
    return 6
bank_addit_df['age'] = bank_addit_df['age'].apply(map_age)


# Поработаем с признаком 'job' (работа):
LabelEncoder(bank_addit_df, 'job') # применим кодирование LabelEncoder


# Поработаем с признаком 'marital' (семейное положение):
LabelEncoder(bank_addit_df, 'marital') # применим кодирование LabelEncoder


# Поработаем с признаком 'education' (образование):
LabelEncoder(bank_addit_df, 'education') # применим кодирование LabelEncoder


# Поработаем с признаком 'default' (еcть ли уже кредит?):
bank_addit_df.default.replace(['no', 'yes', 'unknown'], [0, 1, 2], inplace=True) # применим кодирование через метод replace()


# Поработаем с признаком 'loan' (есть ли кредит для личных целей?):
bank_addit_df.loan.replace(['no', 'yes', 'unknown'], [0, 1, 2], inplace=True) # применим кодирование через метод replace()


# Поработаем с признаком 'housing' (еcть ли кредит на недвижимость?):
bank_addit_df.housing.replace(['no', 'yes', 'unknown'], [0, 1, 2], inplace=True) # применим кодирование через метод replace()


# Поработаем с признаком 'contact' (контактный вид связи):
LabelEncoder(bank_addit_df, 'contact') # применим кодирование LabelEncoder


# Поработаем с признаком 'month' ():
def month_replacer(month):
  month_dct = {'jan': 1,'feb': 2,'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
  return month_dct[month]

bank_addit_df.loc[:, 'month'] = bank_addit_df.month.apply(lambda x: month_replacer(x) )

# for item in bank_addit_df["month"]:
#     month_replacer(item)

# Поработаем с признаком 'day_of_week' ():
def day_replacer(day):
  month_dct = {'mon': 1,'tue': 2,'wed': 3, 'thu': 4, 'fri': 5, 'sat': 6, 'sun': 7}
  return month_dct[day]

bank_addit_df.loc[:, 'day_of_week'] = bank_addit_df.day_of_week.apply(lambda x: day_replacer(x))

# for item in bank_addit_df["day_of_week"]:
#     day_replacer(item)

# Поработаем с признаком 'poutcome' (результат предыдущей маркетинговой кампании):
bank_addit_df.poutcome.replace(['failure', 'success','nonexistent'], [0, 1, 2], inplace=True) # применим кодирование через метод replace()


# Поработаем с признаком 'y' (подписал ли клиент срочный вкла):
bank_addit_df.y.replace(['no', 'yes'], [0, 1], inplace=True) # применим кодирование через метод replace()




bank_addit_df

<ipython-input-13-47c4e83afa0f>:53: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  bank_addit_df.loc[:, 'month'] = bank_addit_df.month.apply(lambda x: month_replacer(x) )
<ipython-input-13-47c4e83afa0f>:63: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  bank_addit_df.loc[:, 'day_of_week'] = bank_addit_df.day_of_week.apply(lambda x: day_replacer(x))


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,6,3,1,0,0,0,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
1,6,7,1,3,2,0,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
2,6,7,1,3,0,1,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
3,6,0,1,1,0,0,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
4,6,7,1,3,0,0,1,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,6,5,1,5,0,1,0,0,11,5,...,1,999,0,2,-1.1,94.767,-50.8,1.028,4963.6,1
41184,6,1,1,5,0,0,0,0,11,5,...,1,999,0,2,-1.1,94.767,-50.8,1.028,4963.6,0
41185,6,5,1,6,0,1,0,0,11,5,...,2,999,0,2,-1.1,94.767,-50.8,1.028,4963.6,0
41186,6,9,1,5,0,0,0,0,11,5,...,1,999,0,2,-1.1,94.767,-50.8,1.028,4963.6,1


In [14]:
bank_addit_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,6,3,1,0,0,0,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
1,6,7,1,3,2,0,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
2,6,7,1,3,0,1,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
3,6,0,1,1,0,0,0,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0
4,6,7,1,3,0,0,1,1,5,1,...,1,999,0,2,1.1,93.994,-36.4,4.857,5191.0,0


In [15]:
bank_addit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  int64  
 2   marital         41188 non-null  int64  
 3   education       41188 non-null  int64  
 4   default         41188 non-null  int64  
 5   housing         41188 non-null  int64  
 6   loan            41188 non-null  int64  
 7   contact         41188 non-null  int64  
 8   month           41188 non-null  int64  
 9   day_of_week     41188 non-null  int64  
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  int64  
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

# 5. Построим модель Логистической регрессии

In [16]:
X = bank_addit_df.drop('y', axis=1)
y = bank_addit_df.y

In [17]:
# Проверим распределение классов:
y.value_counts()

0    36548
1     4640
Name: y, dtype: int64

Обучите обычную логистическую регрессию на этом датасете и выведите classification report.



In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y) # параметр stratify делает подвыборки с равномерным (по возможности) распределением классов
pipe = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression())])
pipe.fit(X_train, y_train)
ypred_train = pipe.predict(X_train)
ypred_test = pipe.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95     28763
           1       0.41      0.66      0.50      2128

    accuracy                           0.91     30891
   macro avg       0.69      0.80      0.73     30891
weighted avg       0.93      0.91      0.92     30891
               precision    recall  f1-score   support

           0       0.97      0.93      0.95      9567
           1       0.42      0.66      0.51       730

    accuracy                           0.91     10297
   macro avg       0.69      0.80      0.73     10297
weighted avg       0.93      0.91      0.92     10297



Какие выводы можете сделать на основании метрик?

А теперь давайте применим особую магию с class_weight.
class_weight="balanced"

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y) # параметр stratify делает подвыборки с равномерным (по возможности) распределением классов
pipe = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression(class_weight="balanced"))])
pipe.fit(X_train, y_train)
ypred_train = pipe.predict(X_train)
ypred_test = pipe.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.85      0.98      0.91     23916
           1       0.85      0.43      0.57      6975

    accuracy                           0.85     30891
   macro avg       0.85      0.70      0.74     30891
weighted avg       0.85      0.85      0.83     30891
               precision    recall  f1-score   support

           0       0.86      0.98      0.91      7966
           1       0.88      0.44      0.58      2331

    accuracy                           0.86     10297
   macro avg       0.87      0.71      0.75     10297
weighted avg       0.86      0.86      0.84     10297

